In [1]:
import numpy as np
import pandas as pd
import graph_tool as gt
from graph_tool import generation

from helper.graph_helper import extract_graph_features, get_neighbors, create_spl_matrix, prepare_compound_features, create_compound_summary_statistics
import joblib

def create_feature_matrix(ppi: gt.Graph, nodes: list, drug_targets: pd.DataFrame, randomization: bool = False, random_state: None or int = None, mercator_df: pd.DataFrame = None):
    if randomization:
        if random_state is not None:
            np.random.seed(random_state)
            gt.seed_rng(random_state)

        generation.random_rewire(ppi, 'configuration', n_iter=1)
        np.random.shuffle(nodes)

    cov_targets = pd.read_csv("../../../0_data/sars_cov2/sars_cov2_targets.txt", header=None)[0].values.tolist()
    cov_targets = list(set(cov_targets).intersection(set(nodes)))

    ppi_df = extract_graph_features(ppi, nodes)

    if mercator_df is not None:
        ppi_df = pd.concat([ppi_df, mercator_df], axis=1)

    if not randomization:
        ppi_df.to_csv("ppi_df.csv", index=True)
    else:
        ppi_df.to_csv(f'ppi_df_randnet_{random_state}.csv', index=True)


    drug_targets['Target'] = drug_targets['Target'].apply(lambda x: ", ".join(set(ppi_df.index).intersection(set(str(x).split(", ")))))
    drug_targets = drug_targets[drug_targets['Target'] != '']
    drug_targets['Neighbor'] = drug_targets['Target'].apply(lambda x: get_neighbors(ppi, x, nodes) if isinstance(x, str) else np.nan)

    features = prepare_compound_features(drug_targets, ppi_df)
    features.loc[:, 'n_targets'] = features['Target'].apply(lambda x: len(x.split(', ')) if isinstance(x, str) else 0)
    features.loc[:, 'n_cov_targets'] = features['Target'].apply(
        lambda x: len(set(str(x).split(', ')).intersection(set(cov_targets))))
    features.loc[:, 'n_neighbors'] = features['Neighbor'].apply(lambda x: len(x.split(', ')) if isinstance(x, str) else 0)

    spl_df = create_spl_matrix(ppi, nodes)
    max_spl = spl_df.max().max()
    spl_df.loc[:, 'mean_spl'] = spl_df.mean(axis=1, skipna=True, numeric_only=True)
    spl_covidome_df = spl_df.loc[:, spl_df.index.intersection(cov_targets)]
    spl_covidome_df['mean_spl_to_covidome'] = spl_covidome_df.mean(axis=1)
    spl_covidome_df['min_spl_to_covidome'] = spl_covidome_df.min(axis=1)
    spl_covidome_df['mean_spl_to_covidome'] = spl_covidome_df['mean_spl_to_covidome'].fillna(max_spl)
    spl_covidome_df['min_spl_to_covidome'] = spl_covidome_df['min_spl_to_covidome'].fillna(max_spl)
    spl_features = spl_covidome_df.loc[:, ['mean_spl_to_covidome', 'min_spl_to_covidome']]
    for col in spl_features.columns:
        features = pd.concat([features, create_compound_summary_statistics(drug_targets, spl_features, col, aim='Target').drop(
            ['Target', 'Neighbor'], axis=1)], join='inner', axis=1)
    features = features.replace({'': np.nan, 'nan': np.nan})

    return features

In [ ]:
target_df = pd.read_csv('../../../0_data/targets/targets.csv', index_col=0)

drugs = {}
for target_set in target_df.columns:
    t = target_df.loc[:, target_set].to_frame()
    t.columns = ['Target']
    drugs[target_set] = t

network_dict = joblib.load(f'../../../1_ppi_networks/large_ppi.sav')
ppi_network = network_dict['network']
ppi_nodes = network_dict['nodes']

structure_features = pd.read_csv('../../../0_data/structure/structure_features.csv', index_col=0)
structure_features = structure_features.loc[:, ~structure_features.columns.str.startswith('Defined')]
structure_features = structure_features.loc[:, ~structure_features.columns.str.startswith('Undefined')]
structure_features = structure_features.drop(['IsotopeAtomCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount'], axis=1, errors="ignore")

features = {}


for target_set in drugs.keys():
    for i in range(5):
        print(target_set + " " + str(i))
        network_features = create_feature_matrix(ppi=ppi_network, nodes=ppi_nodes, drug_targets=drugs[target_set], mercator_df=None, randomization=True, random_state=i)

        features_combined = pd.concat([structure_features, network_features], axis=1, join='inner')
        features_combined = features_combined[features_combined['n_targets'] >= 1]
        features_combined = features_combined.loc[:, ~features_combined.columns.duplicated()]
        features_combined.to_csv(f'features_{target_set}_randnet_{i}.csv')

BDB Target 0


/var/folders/5y/sgsvd71n7yb19_7wd6v8gvh00000gn/T/ipykernel_30624/1500648631.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug_targets['Neighbor'] = drug_targets['Target'].apply(lambda x: get_neighbors(ppi, x, nodes) if isinstance(x, str) else np.nan)


BDB Target 1
BDB Target 2
BDB Target 3
BDB Target 4
DC Target 0
DC Target 1
DC Target 2
DC Target 3
DC Target 4
DRH Target 0
DRH Target 1
DRH Target 2
DRH Target 3
DRH Target 4
TTD Target 0
TTD Target 1
TTD Target 2
TTD Target 3
TTD Target 4
Merged Target 0
Merged Target 1
Merged Target 2
Merged Target 3
